In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"praneethratna","key":"e8df1c59c80efe88e1e7a9bd907ed551"}'}

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d aiswaryaramachandran/hindienglish-corpora

 65% 9.00M/13.9M [00:00<00:00, 22.6MB/s]
100% 13.9M/13.9M [00:00<00:00, 31.5MB/s]


In [ ]:
import os
os.listdir()

['.config', 'hindienglish-corpora.zip', 'sample_data']

In [ ]:
import shutil
shutil.unpack_archive('hindienglish-corpora.zip')

In [ ]:
!mv '/content/Hindi_English_Truncated_Corpus.csv' '/content/hineng.csv'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Embedding , LSTM , Flatten , RepeatVector , TimeDistributed
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from keras.models import load_model

In [ ]:
df = pd.read_csv('hineng.csv').astype('str')
corpora = df[df['source'] == 'ted']
input_statements = list(df['english_sentence'])[:10000]
output_statements = list(df['hindi_sentence'])[:10000]
xtrain , xtest , ytrain , ytest = train_test_split(input_statements,output_statements,test_size = 0.2)

In [ ]:
len(input_statements)

10000

In [ ]:
import re
def text_preprocessing(statements):
  preprocessed_text = list()
  for statement in statements:
    statement = re.sub(r'[()]|[0-9]','',statement)
    words = statement.split()
    words = [word for word in words if len(word) > 1 ]
    words = [word.lower() for word in words]
    temp = ' '.join(words)
    preprocessed_text.append(temp)    
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(preprocessed_text)
  word_index = tokenizer.word_index
  return tokenizer , word_index

In [ ]:
input_tokenizer , input_index = text_preprocessing(input_statements)
output_tokenizer , output_index = text_preprocessing(output_statements)
trainx = input_tokenizer.texts_to_sequences(xtrain)
trainy = output_tokenizer.texts_to_sequences(ytrain)
testx = input_tokenizer.texts_to_sequences(xtest)
testy = output_tokenizer.texts_to_sequences(ytest)

In [ ]:
input_vocab = len(input_index)+1
output_vocab = len(output_index)+1

In [ ]:
input_max_length =40
output_max_length = 40
padded_trainx = pad_sequences(trainx,maxlen = input_max_length,padding='post')
padded_trainy = pad_sequences(trainy,maxlen = output_max_length,padding='post')
padded_trainy = padded_trainy.reshape(padded_trainy.shape[0],padded_trainy.shape[1],1)

In [ ]:
input_max_length

40

In [ ]:
model = Sequential()
model.add(Embedding(input_vocab,256,input_length=input_max_length,mask_zero = True))
model.add(LSTM(256,dropout = 0.3))
model.add(RepeatVector(output_max_length))
model.add(LSTM(256,return_sequences=True,dropout = 0.3))
model.add(TimeDistributed(Dense(output_vocab,activation='softmax')))
cp_path = 'model.save'
callback = ModelCheckpoint(cp_path,monitor='loss',mode='min')
model.compile(optimizer=RMSprop(learning_rate = 0.001),loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

In [ ]:
history = model.fit(padded_trainx,padded_trainy,epochs = 50,batch_size = 256,verbose = 1,callbacks = [callback])

Epoch 1/50
32/32 [==============================] - 21s 314ms/step - loss: 4.7330 - acc: 0.5865


INFO:tensorflow:Assets written to: model.save/assets


INFO:tensorflow:Assets written to: model.save/assets


Epoch 2/50
32/32 [==============================] - 10s 316ms/step - loss: 3.4129 - acc: 0.6067


INFO:tensorflow:Assets written to: model.save/assets


...

INFO:tensorflow:Assets written to: model.save/assets


INFO:tensorflow:Assets written to: model.save/assets


Epoch 50/50
32/32 [==============================] - 11s 335ms/step - loss: 2.5316 - acc: 0.6239


INFO:tensorflow:Assets written to: model.save/assets


INFO:tensorflow:Assets written to: model.save/assets


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 256)           4529920   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 40, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 256)           525312    
_________________________________________________________________
time_distributed (TimeDistri (None, 40, 21486)         5521902   
Total params: 11,102,446
Trainable params: 11,102,446
Non-trainable params: 0
_________________________________________________________________


In [ ]:
padded_testx = pad_sequences(testx,maxlen = input_max_length,padding='post')
padded_testy = pad_sequences(testy,maxlen = output_max_length,padding='post')
padded_testy = padded_testy.reshape(padded_testy.shape[0],padded_testy.shape[1],1)

In [ ]:
model.evaluate(padded_testx,padded_testy)

63/63 [==============================] - 4s 37ms/step - loss: 3.4946 - acc: 0.5934


[3.494633197784424, 0.5933625102043152]